# Visualization of Predictions
Export predictions as colored tables in html format

In [199]:
%matplotlib widget
import os, geopandas, folium, requests, matplotlib, folium.plugins, branca, plotly
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import utils


In [200]:
PREDICTION_DIR = r"predictions"

START_DATE = (datetime.today() - timedelta(days=1)).strftime('%Y-%m-%d') # 7 days before today
END_DATE = datetime.today().strftime('%Y-%m-%d')

POLLUTANTS = ["SO2", "NOy", "CO", "C6H6", "PM10", "PM2.5", "Pb", "O3"]

In [201]:
filename = "predictions_station_Kallio_2.csv"

df = pd.read_csv(PREDICTION_DIR + "/" + filename)
df.head()

,Date,Predicted_PM2.5,Predicted_PM10,Predicted_SO2,Predicted_O3
0,2025-10-01 00:00:00,2.997679,7.352388,1.016420,79.542990
1,2025-10-01 01:00:00,3.920300,9.046658,1.026561,79.435684
2,2025-10-01 02:00:00,4.753071,10.423732,1.048105,79.373160
3,2025-10-01 03:00:00,5.432879,11.471492,1.081256,79.315170
4,2025-10-01 04:00:00,5.965568,12.220298,1.123373,79.266610


Slight preprocessing of prediction data 

In [202]:
def preprocess(df):
    df["Date"] = pd.to_datetime(df["Date"])
    df = df[(df["Date"] >= START_DATE) & \
                    (df["Date"] <= END_DATE)]                  # Filter to a smaller range of dates
    return df

df = preprocess(df)

Generate html prediction table as an example

In [203]:
def make_prediction_table(df, filename, prediction_dir):
    cell_colors = [["white" for i in range(len(df))]]
    cell_values = [df["Date"].dt.strftime("%H:%M").to_list()]


    for column in df.columns:
        for pollutant in POLLUTANTS:
            if pollutant in column: # if the column name contains pollutant name (i.e. "PM2.5")
                
                cell_values.append(df[column].to_list())
                
                cmap = utils.cmaps[pollutant]
                color_mapped_series = df[column].apply(cmap)
                color_mapped_list = color_mapped_series.str[:-2].to_list() # remove last 2 chars which are alpha channel
                cell_colors.append(color_mapped_list)
                


    fig = go.Figure(data=[go.Table(
        header=dict(values=df.columns,
                    fill_color="paleturquoise"
                    ),
        cells=dict(values=cell_values,
                    fill_color=cell_colors
                    ),
        )
    ])

    fig.show()
    fig.write_html(f"{prediction_dir}/{os.path.splitext(filename)[0]}.html") # remove ".csv" from filename

make_prediction_table(df, filename, PREDICTION_DIR)

Generate html prediction tables for all csv files in the predictions folder

In [204]:
dir_predictions = os.path.join(os.getcwd(), PREDICTION_DIR)

for root, _, files in os.walk(dir_predictions): # scans through data directory recursively
    for filename in files:
        if not filename.endswith(".csv"):
            continue
        df = pd.read_csv(os.path.join(root, filename))
        df = preprocess(df)
        make_prediction_table(df, filename, PREDICTION_DIR)